In [ ]:
!pip install git+https://github.com/huggingface/transformers sentencepiece datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-mk4izm0u
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-mk4izm0u
  Resolved https://github.com/huggingface/transformers to commit 656e869a4523f6a0ce90b3aacbb05cc8fb5794bb
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 KB 25.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 KB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 83.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212

In [ ]:
!pip install wikipedia

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11696 sha256=47115a3a8ae31aac45f770a60eb380d4dcaa6e86b71a5e2c53a6b4af1a7fefc1
  Stored in directory: /root/.cache/pip/wheels/c2/46/f4/caa1bee71096d7b0cdca2f2a2af45cacf35c5760bee8f00948
Successfully built wikipedia


In [ ]:
from transformers import pipeline

In [ ]:
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

In [ ]:
import re

import wikipedia
from urllib.request import urlopen
from bs4 import BeautifulSoup

# Get a Wikipedia page by its title
wiki = wikipedia.page('Sceloporus_virgatus')

# Get a Wikipedia page by its url and make a soup
source = urlopen(wiki.url).read()
soup = BeautifulSoup(source,'lxml')

# page will contain the list of sections in the page, as delimited by html headlines
# Add a list with the page title as the first element and the summary as the second
page = [[soup.find('h1').get_text(), wiki.summary]]

for header in soup.find_all(['h2', 'h3']):
    header_name = header.get_text().replace('[edit]', '')
    if header.get_text() == 'Contents':
        continue
    if 'References' in header.get_text():
        break
    # Every element in the sections list will be a list with the name of the
    # headline as the first element, followed by the paragraphs
    section = [header_name]
    for elem in header.next_siblings:
        # Stop at next header
        if elem.name and elem.name.startswith('h'):
            break
        if elem.name == 'p':
            # re.sub eliminate references
            paragraph = re.sub(r'\[.*?\]+', '', elem.get_text())\
                .replace('\n', '')\
                .replace(u'\xa0', ' ')
            section.append(paragraph)
    page.append(section)

In [ ]:
for section in page:
    if len(section) > 1:
        for paragraph in section[1:]:
            lenght = len(paragraph.split())
            summary = summarizer(paragraph, max_length=lenght, min_length=10, do_sample=False)
            print(summary[0]['summary_text'])

The striped plateau lizard is a species of lizard within the genus Sceloporus. It originates from the northern Sierra Madre Occidental and is relatively small in size, measuring less than 72 mm (2.8 in)
"Virgatus" is a Latin word, and it signifies "of twigs" and "rodlike" This term represents the distinct, light
Sceloporus virgatus has an average body mass of 9.35 grams (0.330 oz) The striped plateau lizard has a pair of whiteish-yellow stripes on the dorsolateral sides of its body.
The striped plateau lizards are sexually dimorphic. Males are smaller in size than females, measuring to less than 61 mm (2.4 in) in length.
A curved cornea is an image-forming lens due to the difference in refractive indices between air and corneal tissue. In land vertebrates, many species have lenses, but for these species, the lenses are flatter and weaker.
The striped plateau lizard has a tympanic middle ear. This is a cavity between the outer ear and the inner ear, and it is filled with air. When audit

In [ ]:
from transformers import SpeechT5Processor, SpeechT5ForTextToSpeech, SpeechT5HifiGan
from datasets import load_dataset
import torch
import soundfile as sf

processor = SpeechT5Processor.from_pretrained("microsoft/speecht5_tts")
model = SpeechT5ForTextToSpeech.from_pretrained("microsoft/speecht5_tts")
vocoder = SpeechT5HifiGan.from_pretrained("microsoft/speecht5_hifigan")

Generating validation split: 0 examples [00:00, ? examples/s]

Dataset cmu-arctic-xvectors downloaded and prepared to /root/.cache/huggingface/datasets/Matthijs___cmu-arctic-xvectors/default/0.0.1/a62fea1f9415e240301ea0042ffad2a3aadf4d1caa7f9a8d9512d631723e781f. Subsequent calls will reuse this data.


In [ ]:
inputs = processor(text=summary[0]['summary_text'], return_tensors="pt")

# load xvector containing speaker's voice characteristics from a dataset
embeddings_dataset = load_dataset("Matthijs/cmu-arctic-xvectors", split="validation")
speaker_embeddings = torch.tensor(embeddings_dataset[7306]["xvector"]).unsqueeze(0)

speech = model.generate_speech(inputs["input_ids"], speaker_embeddings, vocoder=vocoder)

sf.write("speech.wav", speech.numpy(), samplerate=16000)